In [1]:
!pip install tensorflow tensorflow-hub

In [2]:
import tensorflow_hub as hub

model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

In [3]:
!pip install opencv-python-headless

In [10]:
import cv2
import numpy as np
import tensorflow as tf

def load_img_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def run_ssd_on_frame(model, frame):
    # Convert frame to RGB and resize it
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, (300, 300))

    # The model expects a uint8 tensor, so no need for normalization to [0, 1]
    # Convert to uint8 data type
    frame_uint8 = frame_resized.astype(np.uint8)

    # Create a batch of one (batch size, height, width, channels)
    input_tensor = tf.convert_to_tensor([frame_uint8], dtype=tf.uint8)

    # Run the frame through the SSD model
    detections = model(input_tensor)
    return detections

In [13]:
vehicle_class_id = 3
min_score_thresh = 0.1  # Minimum score threshold for a detection to be considered
confidence_scores = []  # Store confidence scores

cap = cv2.VideoCapture('/content/video/cars.mp4')  # Update with your video path
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter('/content/processed_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detections = run_ssd_on_frame(model, frame)

    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int64)
    scores = detections['detection_scores'][0].numpy()

    for score, class_id in zip(scores, classes):
        if class_id == vehicle_class_id and score >= min_score_thresh:
            confidence_scores.append(score)
            # Optional: Add code to draw boxes and scores on the frame

    out.write(frame)

cap.release()
out.release()

In [14]:
import pandas as pd

df = pd.DataFrame(confidence_scores, columns=['Confidence Score'])
csv_path = '/content/vehicle_confidence_scores.csv'
df.to_csv(csv_path, index=False)

mean_confidence = df['Confidence Score'].mean()
print(f"Mean Confidence Score: {mean_confidence}")

Mean Confidence Score: 0.5146584510803223
